In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import sys

sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
experiment_dir = "../experiments/mlp_alexnet_bo_20"
meta_log, hyper_log = load_result_logs(experiment_dir)

In [ ]:
from utils.helper import get_encoding_data
from encoding_models import EncoderFitter
import shutil

config_keys = ['dropout', 'hidden_act', 'learning_rate', 'num_hidden_layers',
               'num_hidden_units', 'optimizer_class', 'w_decay']

for e_id in meta_log.eval_ids:
    best_id = np.argmax(meta_log[e_id].stats.best_bo_score.mean)
    best_config = {}
    for c_key in config_keys:
        if type(meta_log[e_id].stats[c_key]) == list:
            config_value = meta_log[e_id].stats[c_key][0][best_id].decode()
        else:
            config_value = meta_log[e_id].stats[c_key].mean[best_id]
        best_config[c_key] = config_value
    best_layer = meta_log[e_id].stats['layer_id'][0][best_id].decode()
    #print(model_config, layer_id)
    
    subject_id = hyper_log[hyper_log.run_id == e_id].subject_id.iloc[0]
    roi_type = hyper_log[hyper_log.run_id == e_id].roi_type.iloc[0]
    
    X, y, X_test = get_encoding_data(fmri_dir="../data/participants_data_v2021",
                                     activations_dir='../data/features/alexnet/pca_100',
                                     layer_id=best_layer,
                                     subject_id=subject_id,
                                     roi_type=roi_type)
    fitter = EncoderFitter("mlp_network", 10,
                           X, y, X_test)
    model_params, y_pred = fitter.predict_on_test(best_config)
    dirpath = "../" + meta_log[e_id].meta.experiment_dir + "extra/"
    save_fname = dirpath + subject_id + "_" + roi_type + "_" + best_layer + "_test.npy"
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
    os.makedirs(dirpath)
    np.save(save_fname , y_pred)
    print(e_id, save_fname)

# Get submission ready based on `experiment_dir`

In [22]:
track = "full_track"
if track == 'full_track': ROIs = ['WB']
else: ROIs = ['LOC','FFA','STS','EBA','PPA','V1','V2','V3','V4']

all_subjects = ['sub01', 'sub02', 'sub03', 'sub04', 'sub05',
                'sub06', 'sub07', 'sub08', 'sub09', 'sub10']

# Load in individual test npy files (subject/roi)
results = {}
for ROI in ROIs:
    ROI_results = {}
    for sub in all_subjects:
        hyper_sub = hyper_log[hyper_log.subject_id == sub]
        hyper_sub_roi = hyper_sub[hyper_sub.roi_type == ROI]
        experiment_dir = hyper_sub_roi.experiment_dir.iloc[0]
        mypath = os.path.join("../" + experiment_dir, "extra")
        try:
            sub_roi_layer = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
            ROI_result = np.load(mypath + "/" + sub_roi_layer[0])
            ROI_results[sub] = ROI_result
        except:
            print(mypath)
            continue
    results[ROI] = ROI_results
results

{'WB': {'sub01': array([[ 0.10600184,  0.05239032,  0.03200618, ..., -0.06142227,
           0.01724791,  0.05402327],
         [-0.07378204, -0.16720746, -0.07610022, ...,  0.03791884,
          -0.01637674, -0.12163191],
         [ 0.07670888,  0.12061948,  0.04559329, ..., -0.02356838,
          -0.00022967,  0.01224203],
         ...,
         [-0.1673174 , -0.05288852, -0.10772121, ..., -0.05722642,
          -0.01200844, -0.03962917],
         [-0.2047551 , -0.13111879, -0.091181  , ..., -0.00479157,
          -0.0189536 , -0.02227664],
         [ 0.08184939,  0.09494755,  0.08184316, ...,  0.01960986,
           0.02147039,  0.06411641]], dtype=float32),
  'sub02': array([[-0.05975054,  0.02410898,  0.01485435, ..., -0.22736111,
           0.02520351,  0.02888802],
         [-0.17503707, -0.18081945, -0.1422095 , ...,  0.09893327,
          -0.10148329, -0.00955446],
         [-0.04660885,  0.01374516, -0.0507448 , ...,  0.03526258,
          -0.09712346, -0.06446679],
         

In [27]:
# Create model_submission sub directory
model_name = "2021-05-31_mlp_alexnet_bo_20"
sub_dir = f"../data/submissions/{model_name}/{track}/"
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)
    
from utils.helper import save_dict
save_dict(results, sub_dir + track + ".pkl")

In [29]:
import zipfile

def zip(src, dst):
    zf = zipfile.ZipFile(dst, "w", zipfile.ZIP_DEFLATED)
    abs_src = os.path.abspath(src)
    for dirname, subdirs, files in os.walk(src):
        for filename in files:
            absname = os.path.abspath(os.path.join(dirname, filename))
            arcname = absname[len(abs_src) + 1:]
            if arcname in ["mini_track.pkl", "full_track.pkl"]:
                zf.write(absname, arcname)
    zf.close()

zip(sub_dir, sub_dir + track + ".zip")